## 1. Tools for text processing
<p><img style="float: right ; margin: 5px 20px 5px 10px; width: 45%" src="https://assets.datacamp.com/production/project_38/img/Moby_Dick_p510_illustration.jpg"> </p>
<p>What are the most frequent words in Herman Melville's novel, Moby Dick, and how often do they occur?</p>
<p>In this notebook, we'll scrape the novel <em>Moby Dick</em> from the website <a href="https://www.gutenberg.org/">Project Gutenberg</a> (which contains a large corpus of books) using the Python package <code>requests</code>. Then we'll extract words from this web data using <code>BeautifulSoup</code>. Finally, we'll dive into analyzing the distribution of words using the Natural Language ToolKit (<code>nltk</code>) and <code>Counter</code>.</p>
<p>The <em>Data Science pipeline</em> we'll build in this notebook can be used to visualize the word frequency distributions of any novel that you can find on Project Gutenberg. The natural language processing tools used here apply to much of the data that data scientists encounter as a vast proportion of the world's data is unstructured data and includes a great deal of text.</p>
<p>Let's start by loading in the three main Python packages we are going to use.</p>

In [167]:
# Importing requests, BeautifulSoup, nltk, and Counter
# ... YOUR CODE FOR TASK 1 ...
import requests
from bs4 import BeautifulSoup
import nltk
from collections import Counter

## 2. Request Moby Dick
<p>To analyze Moby Dick, we need to get the contents of Moby Dick from <em>somewhere</em>. Luckily, the text is freely available online at Project Gutenberg as an HTML file: https://www.gutenberg.org/files/2701/2701-h/2701-h.htm .</p>
<p><strong>Note</strong> that HTML stands for Hypertext Markup Language and is the standard markup language for the web.</p>
<p>To fetch the HTML file with Moby Dick we're going to use the <code>request</code> package to make a <code>GET</code> request for the website, which means we're <em>getting</em> data from it. This is what you're doing through a browser when visiting a webpage, but now we're getting the requested page directly into Python instead. </p>

In [169]:
# Getting the Moby Dick HTML 
r = requests.get('https://s3.amazonaws.com/assets.datacamp.com/production/project_147/datasets/2701-h.htm')

# Setting the correct text encoding of the HTML page
r.encoding = 'utf-8'

# Extracting the HTML from the request object
html = r.text

# Printing the first 2000 characters in html
# ... YOUR CODE FOR TASK 3 ...
html[0:2000]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "GET /assets.datacamp.com/production/project_147/datasets/2701-h.htm HTTP/1.1" 200 1501037


'<?xml version="1.0" encoding="utf-8"?>\r\n\r\n<!DOCTYPE html\r\n   PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"\r\n   "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd" >\r\n\r\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en">\r\n  <head>\r\n    <title>\r\n      Moby Dick; Or the Whale, by Herman Melville\r\n    </title>\r\n    <style type="text/css" xml:space="preserve">\r\n\r\n    body { background:#faebd0; color:black; margin-left:15%; margin-right:15%; text-align:justify }\r\n    P { text-indent: 1em; margin-top: .25em; margin-bottom: .25em; }\r\n    H1,H2,H3,H4,H5,H6 { text-align: center; margin-left: 15%; margin-right: 15%; }\r\n    hr  { width: 50%; text-align: center;}\r\n    .foot { margin-left: 20%; margin-right: 20%; text-align: justify; text-indent: -3em; font-size: 90%; }\r\n    blockquote {font-size: 100%; margin-left: 0%; margin-right: 0%;}\r\n    .mynote    {background-color: #DDE; color: #000; padding: .5em; margin-left: 10%; margin-right: 10%; font-family: sans-se

## 3. Get the text from the HTML
<p>This HTML is not quite what we want. However, it does <em>contain</em> what we want: the text of <em>Moby Dick</em>. What we need to do now is <em>wrangle</em> this HTML to extract the text of the novel. For this we'll use the package <code>BeautifulSoup</code>.</p>
<p>Firstly, a word on the name of the package: Beautiful Soup? In web development, the term "tag soup" refers to structurally or syntactically incorrect HTML code written for a web page. What Beautiful Soup does best is to make tag soup beautiful again and to extract information from it with ease! In fact, the main object created and queried when using this package is called <code>BeautifulSoup</code>.</p>

In [171]:
# Creating a BeautifulSoup object from the HTML
soup = BeautifulSoup(html)

# Getting the text out of the soup
text = soup.text

# Printing out text between characters 32000 and 34000
# ... YOUR CODE FOR TASK 3 ...
text[32000:34000]

'r which the beech tree\r\n        extended its branches.” —Darwin’s Voyage of a Naturalist.\r\n      \n\r\n        “‘Stern all!’ exclaimed the mate, as upon turning his head, he saw the\r\n        distended jaws of a large Sperm Whale close to the head of the boat,\r\n        threatening it with instant destruction;—‘Stern all, for your\r\n        lives!’” —Wharton the Whale Killer.\r\n      \n\r\n        “So be cheery, my lads, let your hearts never fail, While the bold\r\n        harpooneer is striking the whale!” —Nantucket Song.\r\n      \n\r\n     “Oh, the rare old Whale, mid storm and gale\r\n     In his ocean home will be\r\n     A giant in might, where might is right,\r\n     And King of the boundless sea.”\r\n      —Whale Song.\r\n\n\n\n\n\n \n\n\n\n\n\r\n      CHAPTER 1. Loomings.\r\n    \n\r\n      Call me Ishmael. Some years ago—never mind how long precisely—having\r\n      little or no money in my purse, and nothing particular to interest me on\r\n      shore, I thought I

## 4. Extract the words
<p>We now have the text of the novel! There is some unwanted stuff at the start and some unwanted stuff at the end. We could remove it, but this content is so much smaller in amount than the text of Moby Dick that, to a first approximation, it is okay to leave it in.</p>
<p>Now that we have the text of interest, it's time to count how many times each word appears, and for this we'll use <code>nltk</code> – the Natural Language Toolkit. We'll start by tokenizing the text, that is, remove everything that isn't a word (whitespace, punctuation, etc.) and then split the text into a list of words.</p>

In [173]:
# Creating a tokenizer
tokenizer = nltk.tokenize.RegexpTokenizer('\w+')

# Tokenizing the text
tokens = tokenizer.tokenize(text)

# Printing out the first 8 words / tokens
# ... YOUR CODE FOR TASK 4 ...
tokens[0:8]

['Moby', 'Dick', 'Or', 'the', 'Whale', 'by', 'Herman', 'Melville']

## 5. Make the words lowercase
<p>OK! We're nearly there. Note that in the above 'Or' has a capital 'O' and that in other places it may not, but both 'Or' and 'or' should be counted as the same word. For this reason, we should build a list of all words in <em>Moby Dick</em> in which all capital letters have been made lower case.</p>

In [175]:
# Create a list called words containing all tokens transformed to lower-case
# ... YOUR CODE FOR TASK 5 ...
words = [word.lower() for word in tokens]
# Printing out the first 8 words / tokens 
# ... YOUR CODE FOR TASK 5 ...
words[0:8]

['moby', 'dick', 'or', 'the', 'whale', 'by', 'herman', 'melville']

## 6. Load in stop words
<p>It is common practice to remove words that appear a lot in the English language such as 'the', 'of' and 'a' because they're not so interesting. Such words are known as <em>stop words</em>. The package <code>nltk</code> includes a good list of stop words in English that we can use.</p>

In [177]:
# Getting the English stop words from nltk
from nltk.corpus import stopwords
sw = stopwords.words('english')

# Printing out the first eight stop words
# ... YOUR CODE FOR TASK 6 ...
sw[0:8]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves']

## 7. Remove stop words in Moby Dick
<p>We now want to create a new list with all <code>words</code> in Moby Dick, except those that are stop words (that is, those words listed in <code>sw</code>).</p>

In [179]:
# Create a list words_ns containing all words that are in words but not in sw
# ... YOUR CODE FOR TASK 7 ...
words_ns = [word for word in words if word not in sw]
# Printing the first 5 words_ns to check that stop words are gone
# ... YOUR CODE FOR TASK 7 ...
words_ns[0:5]

['moby', 'dick', 'whale', 'herman', 'melville']

## 8. We have the answer
<p>Our original question was:</p>
<blockquote>
  <p>What are the most frequent words in Herman Melville's novel Moby Dick and how often do they occur?</p>
</blockquote>
<p>We are now ready to answer that! Let's answer this question using the <code>Counter</code> class we imported earlier.</p>

In [181]:
# Initialize a Counter object from our processed list of words
count = Counter(words_ns)

# Store 10 most common words and their counts as top_ten
top_ten = ...

# Print the top ten words and their counts


Counter({'moby': 91,
         'dick': 91,
         'whale': 1246,
         'herman': 5,
         'melville': 5,
         'body': 111,
         'background': 5,
         'faebd0': 1,
         'color': 7,
         'black': 93,
         'margin': 31,
         'left': 80,
         '15': 6,
         'right': 168,
         'text': 18,
         'align': 7,
         'justify': 4,
         'p': 3,
         'indent': 2,
         '1em': 1,
         'top': 72,
         '25em': 2,
         'bottom': 57,
         'h1': 1,
         'h2': 1,
         'h3': 1,
         'h4': 1,
         'h5': 1,
         'h6': 1,
         'center': 3,
         'hr': 1,
         'width': 4,
         '50': 4,
         'foot': 34,
         '20': 10,
         '3em': 1,
         'font': 8,
         'size': 17,
         '90': 5,
         'blockquote': 1,
         '100': 6,
         '0': 8,
         'mynote': 1,
         'dde': 1,
         '000': 22,
         'padding': 2,
         '5em': 1,
         '10': 10,
         'famil

## 9. The most common word
<p>Nice! Using our variable <code>top_ten</code>, we now have an answer to our original question.</p>
<p>The natural language processing skills we used in this notebook are also applicable to much of the data that Data Scientists encounter as the vast proportion of the world's data is unstructured data and includes a great deal of text. </p>
<p>So, what word turned out to (<em>not surprisingly</em>) be the most common word in Moby Dick?</p>

In [183]:
# What's the most common word in Moby Dick?
most_common_word = 'horse'